In [1]:
import time
import paramiko
import os
import pandas as pd
import sqlite3
# !pip install ipython-sql # for sql magic abilities, i.e. %%sql, etc.
import IPython
import pyodbc

In [2]:
# Capture process start time
MAIN_START_TIME = time.strftime("%Y-%m-%d %H:%M:%S")

print('Process Start time:      ' + MAIN_START_TIME)

Process Start time:      2022-03-08 15:26:59


In [3]:
# Microsoft SQL Server connection
conn = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',
                      server = 'localhost', database = 'databaseName')
cursor = conn.cursor()

In [ ]:
# Display SQL Server database name if connected
cursor.execute("SELECT @@SERVERNAME;")
cursor.fetchone()

In [7]:
sql_ids = '''
select STUDENT_GU, SIS_NUMBER from databaseName.Rev.EPC_STU
'''

ids = pd.read_sql(sql_ids,conn)
print('SQL Synergy production Student ID data is now in dataframe.')

SQL Synergy production Student ID data is now in dataframe.


In [8]:
# close connection to SQL Server
conn.close()
print('SQL Server connection closed...')

SQL Server connection closed...


In [9]:
# Site Credentials into dataframe
sc = pd.read_csv('D:/BCP/Python/SiteCredentials.txt', engine='python')

In [10]:
# define connection parameters and values

#.to_string(index=False) removes dataframe metadata to pass value
host = "files.americanreading.com"
port = 22
username = sc.username[(sc['RowNum'] == 1)].to_string(index=False)
password = sc.password[(sc['RowNum'] == 1)].to_string(index=False)

# Running this in Cygwin64 Terminal will add site to known_hosts file (if not present - usually the case on Windows)
        # (EXAMPLE) ssh-keyscan -H files.americanreading.com >> /cygdrive/d/BCP/Python/known_hosts
known_hosts_dir = "D:/BCP/Python/known_hosts"

In [11]:
# passing Paramiko's SSHClient capabilities to "ssh" alias
ssh = paramiko.SSHClient()
print("SSHClient loaded...  " + str(ssh))

SSHClient loaded...  <paramiko.client.SSHClient object at 0x0000018E7D608DD8>


In [12]:
#load host keys from known_hosts file
ssh.load_host_keys(os.path.expanduser(known_hosts_dir))
print("known_hosts loaded...")

known_hosts loaded...


In [13]:
# connect to sftp server
#ssh.connect(host, port, username, password )
#print('...connected to ' + host)

ssh.connect(hostname="transfers.americanreading.com", port=22
            , username = sc.username[(sc['RowNum'] == 2)].to_string(index=False)
            , password = sc.password[(sc['RowNum'] == 2)].to_string(index=False) ) 
    

In [14]:
# pass SFTP abilities to "sftp" alias
sftp = ssh.open_sftp()
print("SFTPClient loaded...  " + str(sftp))

SFTPClient loaded...  <paramiko.sftp_client.SFTPClient object at 0x0000018E7D4676A0>


In [15]:
# lists all remote directories
listofdirectories = sftp.listdir()
print("Directories:     " + str(listofdirectories))

Directories:     ['downloads', 'uploads']


In [16]:
# lists files in specific remote directory
maindirectory = '/downloads'

# get latest file name, from files matching startswith criteria from the explicit directory
latest = 0
latestfile = None

for fileattr in sftp.listdir_attr(maindirectory):
    if fileattr.filename.startswith('20'):# and fileattr.st_mtime > latest: 
 ##commented this out, as files for several days on SFTP have same create time stamp, resulting in first file in series
        latest = fileattr.st_mtime
        latestfile = fileattr.filename
        
# Remote path and latest file        
pathFileName = '/downloads/'+ latestfile
print("Most recent available file:       '"+ pathFileName + "'" )

Most recent available file:       '/downloads/2022-03-07_nati_student-data-export.csv'


In [17]:
# open csv file from sftp server and place its results in "file" placeholder
file = sftp.open(pathFileName)

In [18]:
# place csv file into pandas dataframe
df = pd.read_csv(file)

In [19]:
# Close SFTP connection
ssh.close()
print('...connection to ' + host + ' closed')

...connection to files.americanreading.com closed


In [20]:
#clean up column names, stripping them of extra spaces
df.columns = (df.columns.str.strip().str.replace(' ', '').str.replace('-',''))

In [21]:
# create the database connection
cnn = sqlite3.connect('arc.db')

In [22]:
# load dataframe into database -- if table exists, replace
df.to_sql("arc", cnn, if_exists='replace')
ids.to_sql("ids", cnn, if_exists='replace')

In [23]:
# loal sql module from iPython (Jupyter Notebook code)
%load_ext sql

## Python file (py) needs this as a defined function to represent this code
#load_ext_text = '%load_ext sql'
#def load_ext(load_ext_text):
#    return load_ext

In [24]:
# connect sqlite to database (Jupyter Notebook code)
%sql sqlite:///arc.db

## Python file (py) needs this as a defined function to represent this code   
#connect_to_db_text = '%sql'', ''sqlite:///arc.db'
#def connect_to_db(connect_to_db_text):
#    return connect_to_db

In [25]:
# SQL results into dataframe

query = '''
WITH RECURSIVE
a ( Student_ID, Recorded_At, Scaled_Score, Assessed_Level, Subject_Area, Assessment_Name, School_ID, subscore, [subscore assessed level])
as(
select StudentID            AS Student_ID
  --  ,  strftime('%m/%d/%Y',date(IRLACurrentReportingDate))  AS Recorded_At
    ,  IRLACurrentReportingDate AS Recorded_At
    ,  IRLAScore            AS Scaled_Score
    ,  IRLADataWallTier     AS Assessed_Level
    ,  'ARC'                AS Subject_Area
    ,  'IRLA'               AS Assessment_Name
    ,  SchoolName           AS School_ID
    ,  ''                   AS subscore
    ,  ''                   AS [subscore assessed level]
from arc
where IRLAScore <> "None" and StudentID <> "None" and IRLACurrentReportingDate <> "None"

union

select StudentID            AS Student_ID
   -- ,  strftime('%m/%d/%Y',date(ENILCurrentReportingDate))  AS Recorded_At
    ,  ENILCurrentReportingDate AS Recorded_At
    ,  ENILScore            AS Scaled_Score
    ,  ENILDataWallTier     AS Assessed_Level
    ,  'ARC'                AS Subject_Area
    ,  'ENIL'               AS Assessment_Name
    ,  SchoolName           AS School_ID
    ,  ''                   AS subscore
    ,  ''                   AS [subscore assessed level]
from arc
where ENILScore <> "None" and StudentID <> "None" and ENILCurrentReportingDate <> "None"
)
select distinct
       b.SIS_NUMBER    AS Student_ID
    --,  Recorded_At
    ,  strftime('%m/%d/%Y',date(Recorded_At)) AS Recorded_At
    ,  Scaled_Score
    ,  Assessed_Level
    ,  Subject_Area
    ,  Assessment_Name
    ,  case when School_ID = 'Central Elementary'      then '010' 
            when School_ID = 'El Toyon Elementary'     then '020'
            when School_ID = 'Ira Harbison'            then '030'
            when School_ID = 'John A. Otis Elementary' then '085'
            when School_ID = 'Kimball'                 then '050'
            when School_ID = 'Las Palmas'              then '060'
            when School_ID = 'Lincoln Acres'           then '070'
            when School_ID = 'Olivewood'               then '080'
            when School_ID = 'Palmer Way'              then '090'
            when School_ID = 'Rancho de la Nacion'     then '025'
       else School_ID end  AS School_ID
     , subscore
     , [subscore assessed level]
     , case when Recorded_At between '2021-07-28' and '2021-08-25' then 'Fall'
            when Recorded_At between '2021-10-25' and '2021-10-29' then 'Midfall'
            when Recorded_At between '2021-11-29' and '2021-12-17' then 'Winter'
            when Recorded_At between '2022-02-28' and '2022-03-04' then 'Midwinter'
            when Recorded_At between '2022-04-25' and '2022-05-13' then 'Spring' end as FileName
from   a
join   ids as b
  on   cast(a.Student_ID as varchar) = cast(b.STUDENT_GU as varchar)
'''

sql_df = pd.read_sql_query(query, cnn)

In [26]:
# Dataframe results to csv file
sql_df.to_csv(r'D:\BCP\Panorama\ARC\ARC.csv', index=False ,encoding='utf-8-sig')

In [27]:
# passing Paramiko's SSHClient capabilities to "ssh" alias for new Paramiko instance
ssh2 = paramiko.SSHClient()
print("SSHClient loaded...  " + str(ssh2))


# define connection parameters and values
host2 = "files.panoramaed.com"
port2 = 22
username2 = sc.username[(sc['RowNum'] == 2)].to_string(index=False)
password2 = sc.password[(sc['RowNum'] == 2)].to_string(index=False)

# Running this in Cygwin64 Terminal will add site to known_hosts file (if not present - usually the case on Windows)
        # (EXAMPLE) ssh-keyscan -H files.panoramaed.com >> /cygdrive/d/BCP/Python/known_hosts
known_hosts_dir2 = "D:/BCP/Python/known_hosts"


#load host keys from known_hosts file
ssh2.load_host_keys(os.path.expanduser(known_hosts_dir2))

# connect to sftp server
ssh2.connect(hostname="files.panoramaed.com", port=22
             , username = sc.username[(sc['RowNum'] == 3)].to_string(index=False)
             , password = sc.password[(sc['RowNum'] == 3)].to_string(index=False) )
print('...connected to ' + host2)

SSHClient loaded...  <paramiko.client.SSHClient object at 0x0000018E7D7D9710>
...connected to files.panoramaed.com


In [28]:
# pass SFTP abilities to "sftp" alias
sftp2 = ssh2.open_sftp()

In [29]:
# lists all remote directories
sftp2.listdir()

['2019-2020', '2020-2021', 'Sample Files', '2021-2022']

In [30]:
# lists files in specific remote directory
sftp2.listdir('2020-2021/ARC')

['ARC.csv']

In [31]:
# set remote and local paths
remotepath1 = r"/2021-2022/ARC/Fall.csv"
print("Remote file path:    '" + remotepath1 + "'")
localfile1 = r"D:/BCP/Panorama/ARC/Fall.csv"
print("Local file path:     '" + localfile1 + "'")


remotepath2 = r"/2021-2022/ARC/Midfall.csv"
print("Remote file path:    '" + remotepath2 + "'")
localfile2 = r"D:/BCP/Panorama/ARC/Midfall.csv"
print("Local file path:     '" + localfile2 + "'")


remotepath3 = r"/2021-2022/ARC/Winter.csv"
print("Remote file path:    '" + remotepath3 + "'")
localfile3 = r"D:/BCP/Panorama/ARC/Winter.csv"
print("Local file path:     '" + localfile3 + "'")


remotepath4 = r"/2021-2022/ARC/Midwinter.csv"
print("Remote file path:    '" + remotepath4 + "'")
localfile4 = r"D:/BCP/Panorama/ARC/Midwinter.csv"
print("Local file path:     '" + localfile4 + "'")


remotepath5 = r"/2021-2022/ARC/Spring.csv"
print("Remote file path:    '" + remotepath5 + "'")
localfile5 = r"D:/BCP/Panorama/ARC/Spring.csv"
print("Local file path:     '" + localfile5 + "'")

Remote file path:    '/2021-2022/ARC/Fall.csv'
Local file path:     'D:/BCP/Panorama/ARC/Fall.csv'
Remote file path:    '/2021-2022/ARC/Midfall.csv'
Local file path:     'D:/BCP/Panorama/ARC/Midfall.csv'
Remote file path:    '/2021-2022/ARC/Winter.csv'
Local file path:     'D:/BCP/Panorama/ARC/Winter.csv'
Remote file path:    '/2021-2022/ARC/Midwinter.csv'
Local file path:     'D:/BCP/Panorama/ARC/Midwinter.csv'
Remote file path:    '/2021-2022/ARC/Spring.csv'
Local file path:     'D:/BCP/Panorama/ARC/Spring.csv'


In [32]:
# send local file to remote server
sftp2.put(localfile1, remotepath1, confirm=True)
print('...file ' + localfile1 + ' uploaded successfully to ' + remotepath1 + ' on ' + host2)

sftp2.put(localfile2, remotepath2, confirm=True)
print('...file ' + localfile2 + ' uploaded successfully to ' + remotepath2 + ' on ' + host2)

sftp2.put(localfile3, remotepath3, confirm=True)
print('...file ' + localfile3 + ' uploaded successfully to ' + remotepath3 + ' on ' + host2)

sftp2.put(localfile4, remotepath4, confirm=True)
print('...file ' + localfile4 + ' uploaded successfully to ' + remotepath4 + ' on ' + host2)

sftp2.put(localfile5, remotepath5, confirm=True)
print('...file ' + localfile5 + ' uploaded successfully to ' + remotepath5 + ' on ' + host2)


...file D:/BCP/Panorama/ARC/Fall.csv uploaded successfully to /2021-2022/ARC/Fall.csv on files.panoramaed.com
...file D:/BCP/Panorama/ARC/Midfall.csv uploaded successfully to /2021-2022/ARC/Midfall.csv on files.panoramaed.com
...file D:/BCP/Panorama/ARC/Winter.csv uploaded successfully to /2021-2022/ARC/Winter.csv on files.panoramaed.com
...file D:/BCP/Panorama/ARC/Midwinter.csv uploaded successfully to /2021-2022/ARC/Midwinter.csv on files.panoramaed.com
...file D:/BCP/Panorama/ARC/Spring.csv uploaded successfully to /2021-2022/ARC/Spring.csv on files.panoramaed.com


In [33]:
# close client connection and sftp
sftp.close()
ssh.close()
print('...connection to ' + host + ' closed')

...connection to files.americanreading.com closed


In [34]:
# Capture process start time
MAIN_STOP_TIME = time.strftime("%Y-%m-%d %H:%M:%S")

print('Process Stop time:      ' + MAIN_STOP_TIME)

Process Stop time:      2022-03-08 15:27:50
